API exercises:

Goals with this exercise set:
 - Acquire all people
 - Acquire all starships
 - Acquire all planets
Get the Weather data at 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'

In [1]:
import pandas as pd
import requests

In [2]:
# lets grab stuff the same way we did initally:

In [3]:
# steps that we know that we need to do:
# get the first page of content
# get the url endpoint for the next page of content
# get a dataframe, get our next location

In [4]:
home_url = 'https://swapi.dev/api'

In [5]:
# initial request:
# put the contents of the home page in home_page
home_page = requests.get(home_url).json()

In [6]:
# first endpoint for payload number one:
home_page['people']

'https://swapi.dev/api/people/'

In [7]:
# plug that endpoint into a new requests
payload = requests.get(home_page['people'])

In [8]:
payload.json().keys()

dict_keys(['count', 'next', 'previous', 'results'])

In [9]:
# things that I want from this first payload:
# results -> the stuff that im going to turn into a dataframe
# next -> my next payload endpoint that I need for another page

In [10]:
# lets make a function to automate this for all three datasets:

In [14]:
def get_content(topic_page)->pd.DataFrame:
    '''
    get_content is meant to interact with the star wars api
    at swapi.dev.
    
    under the presumption of the structure of any of the pages
    following the topics under the home page of swapi.dev,
    get_content will traverse the pages of content
    associated with that topic,
    grab all rows of content,
    concatenate those rows of content into a single dataframe
    and return out the complete data set as a complete pandas 
    dataframe
    '''
    # step one:
    # the arg in this is going to be the endpoint of the topic
    # payload will be from the json() version of the response
    print('getting page number 1')
    payload = requests.get(topic_page).json()
    # payload is a dictionary, get the values associated with
    # the next endpoint:
    next_page = payload['next']
    # content is going to be a list of dictionaries
    content = payload['results']
    df = pd.DataFrame(content)
    # under the condition that we still have a next page:
    while next_page is not None:
        # get the next page of content
        payload = requests.get(next_page).json()
        # grab the next endpoint
        print(f'getting page number {next_page[-1]}')
        next_page = payload['next']
        # get the content
        content = payload['results']
        # reassign the dataframe with the new content added
        df = pd.concat(
            [
            df, pd.DataFrame(content)
            ], ignore_index=True)
    return df

In [12]:
# topic page example:
home_page['people']

'https://swapi.dev/api/people/'

In [15]:
import time

In [16]:
t0 = time.time()
people = get_content(home_page['people'])
tx = time.time()
print(f'Data obtained! We aquired the data in {round(((tx - t0) / 60),2)} minutes')

getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6
getting page number 7
getting page number 8
getting page number 9
Data obtained! We aquired the data in 0.53 minutes


In [17]:
t0 = time.time()
starships = get_content(home_page['starships'])
tx = time.time()
print(f'Data obtained! We aquired the data in {round(((tx - t0) / 60),2)} minutes')

getting page number 1
getting page number 2
getting page number 3
getting page number 4
Data obtained! We aquired the data in 0.16 minutes


In [18]:
t0 = time.time()
planets = get_content(home_page['planets'])
tx = time.time()
print(f'Data obtained! We aquired the data in {round(((tx - t0) / 60),2)} minutes')

getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6
Data obtained! We aquired the data in 0.22 minutes


In [19]:
# lets get these dataframes associated with eachother

In [20]:
# how do people link to starships?
# people drive starships I think

In [21]:
starships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    36 non-null     object
 1   model                   36 non-null     object
 2   manufacturer            36 non-null     object
 3   cost_in_credits         36 non-null     object
 4   length                  36 non-null     object
 5   max_atmosphering_speed  36 non-null     object
 6   crew                    36 non-null     object
 7   passengers              36 non-null     object
 8   cargo_capacity          36 non-null     object
 9   consumables             36 non-null     object
 10  hyperdrive_rating       36 non-null     object
 11  MGLT                    36 non-null     object
 12  starship_class          36 non-null     object
 13  pilots                  36 non-null     object
 14  films                   36 non-null     object
 15  created 

In [22]:
# lets look at pilots in starships as a link:
starships.pilots

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4     [https://swapi.dev/api/people/13/, https://swa...
5                                                    []
6     [https://swapi.dev/api/people/1/, https://swap...
7                     [https://swapi.dev/api/people/4/]
8                                                    []
9                                                    []
10                   [https://swapi.dev/api/people/22/]
11    [https://swapi.dev/api/people/1/, https://swap...
12                                                   []
13                                                   []
14                   [https://swapi.dev/api/people/29/]
15                                                   []
16                                                   []
17                                              

In [24]:
# enter pd.DataFrame.explode:
# if we explode out pilots,
# the elements in those lists of those rows
# will expand out into single entries amongst
# multiple rows of a dataframe
# these exploded values link up to the people urls.
starships.explode('pilots').head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,NaN,"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,NaN,"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,NaN,[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/
3,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,NaN,[https://swapi.dev/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-20T21:26:24.783000Z,https://swapi.dev/api/starships/9/
4,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,https://swapi.dev/api/people/13/,"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:59:45.094000Z,2014-12-20T21:23:49.880000Z,https://swapi.dev/api/starships/10/


In [26]:
# left key: people.url

In [27]:
people_and_ships = pd.merge(
left=people,
right=starships.explode('pilots'),
left_on= 'url',
right_on= 'pilots',
how='left',
suffixes=['_people', '_starships']
)

In [28]:
people_and_ships.shape

(93, 34)

In [30]:
people_and_ships.head()

,name_people,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films_people,...,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films_starships,created_starships,edited_starships,url_starships
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,110,1 week,1.0,100,Starfighter,https://swapi.dev/api/people/1/,"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-12T11:19:05.340000Z,2014-12-20T21:23:49.886000Z,https://swapi.dev/api/starships/12/
1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,80000,2 months,1.0,50,Armed government transport,https://swapi.dev/api/people/1/,"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-15T13:04:47.235000Z,2014-12-20T21:23:49.900000Z,https://swapi.dev/api/starships/22/
2,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,150,5 days,1.0,105,Starfighter,https://swapi.dev/api/people/4/,[https://swapi.dev/api/films/1/],2014-12-12T11:21:32.991000Z,2014-12-20T21:23:49.889000Z,https://swapi.dev/api/starships/13/


In [32]:
people.homeworld

0      https://swapi.dev/api/planets/1/
1      https://swapi.dev/api/planets/1/
2      https://swapi.dev/api/planets/8/
3      https://swapi.dev/api/planets/1/
4      https://swapi.dev/api/planets/2/
                    ...                
77    https://swapi.dev/api/planets/59/
78    https://swapi.dev/api/planets/14/
79     https://swapi.dev/api/planets/2/
80    https://swapi.dev/api/planets/60/
81    https://swapi.dev/api/planets/12/
Name: homeworld, Length: 82, dtype: object

In [33]:
# key pairing:
# people.homeworld,
# planets.url
everything = pd.merge(
left=people_and_ships,
right=planets,
how='left',
left_on='homeworld',
right_on='url')

In [34]:
everything.head()

,name_people,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films_people,...,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
2,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
3,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",...,temperate,1 standard,"grassy hills, swamps, forests, mountains",12,4500000000,"[https://swapi.dev/api/people/3/, https://swap...","[https://swapi.dev/api/films/3/, https://swapi...",2014-12-10T11:52:31.066000Z,2014-12-20T20:58:18.430000Z,https://swapi.dev/api/planets/8/
4,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",...,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/


In [35]:
everything.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name_people             93 non-null     object
 1   height                  93 non-null     object
 2   mass                    93 non-null     object
 3   hair_color              93 non-null     object
 4   skin_color              93 non-null     object
 5   eye_color               93 non-null     object
 6   birth_year              93 non-null     object
 7   gender                  93 non-null     object
 8   homeworld               93 non-null     object
 9   films_people            93 non-null     object
 10  species                 93 non-null     object
 11  vehicles                93 non-null     object
 12  starships               93 non-null     object
 13  created_people          93 non-null     object
 14  edited_people           93 non-null     object
 15  url_peop

In [36]:
# if i wanted to re-aggregate to make the content unique by person:
# i could reaggregate the content into a list

In [37]:
# you can implode by reaggregating as a list:
everything.groupby('name_people').agg(list)

,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films_people,species,...,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
name_people,,,,,,,,,,,,,,,,,,,,,
Ackbar,[180],[83],[none],[brown mottle],[orange],[41BBY],[male],[https://swapi.dev/api/planets/31/],[[https://swapi.dev/api/films/3/]],[[https://swapi.dev/api/species/8/]],...,[temperate],[1],"[oceans, reefs, islands]",[100],[27000000000],[[https://swapi.dev/api/people/27/]],[[]],[2014-12-18T11:07:01.792000Z],[2014-12-20T20:58:18.471000Z],[https://swapi.dev/api/planets/31/]
Adi Gallia,[184],[50],[none],[dark],[blue],[unknown],[female],[https://swapi.dev/api/planets/9/],"[[https://swapi.dev/api/films/4/, https://swap...",[[https://swapi.dev/api/species/23/]],...,[temperate],[1 standard],"[cityscape, mountains]",[unknown],[1000000000000],"[[https://swapi.dev/api/people/34/, https://sw...","[[https://swapi.dev/api/films/3/, https://swap...",[2014-12-10T11:54:13.921000Z],[2014-12-20T20:58:18.432000Z],[https://swapi.dev/api/planets/9/]
Anakin Skywalker,"[188, 188, 188]","[84, 84, 84]","[blond, blond, blond]","[fair, fair, fair]","[blue, blue, blue]","[41.9BBY, 41.9BBY, 41.9BBY]","[male, male, male]","[https://swapi.dev/api/planets/1/, https://swa...","[[https://swapi.dev/api/films/4/, https://swap...","[[], [], []]",...,"[arid, arid, arid]","[1 standard, 1 standard, 1 standard]","[desert, desert, desert]","[1, 1, 1]","[200000, 200000, 200000]","[[https://swapi.dev/api/people/1/, https://swa...","[[https://swapi.dev/api/films/1/, https://swap...","[2014-12-09T13:50:49.641000Z, 2014-12-09T13:50...","[2014-12-20T20:58:18.411000Z, 2014-12-20T20:58...","[https://swapi.dev/api/planets/1/, https://swa..."
Arvel Crynyd,[unknown],[unknown],[brown],[fair],[brown],[unknown],[male],[https://swapi.dev/api/planets/28/],[[https://swapi.dev/api/films/3/]],[[]],...,[unknown],[unknown],[unknown],[unknown],[unknown],"[[https://swapi.dev/api/people/20/, https://sw...",[[]],[2014-12-15T12:25:59.569000Z],[2014-12-20T20:58:18.466000Z],[https://swapi.dev/api/planets/28/]
Ayla Secura,[178],[55],[none],[blue],[hazel],[48BBY],[female],[https://swapi.dev/api/planets/37/],"[[https://swapi.dev/api/films/4/, https://swap...",[[https://swapi.dev/api/species/15/]],...,"[temperate, arid, subartic]",[1],"[mountains, valleys, deserts, tundra]",[5],[1500000000],"[[https://swapi.dev/api/people/45/, https://sw...",[[]],[2014-12-20T09:46:25.740000Z],[2014-12-20T20:58:18.481000Z],[https://swapi.dev/api/planets/37/]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wicket Systri Warrick,[88],[20],[brown],[brown],[brown],[8BBY],[male],[https://swapi.dev/api/planets/7/],[[https://swapi.dev/api/films/3/]],[[https://swapi.dev/api/species/9/]],...,[temperate],[0.85 standard],"[forests, mountains, lakes]",[8],[30000000],[[https://swapi.dev/api/people/30/]],[[https://swapi.dev/api/films/3/]],[2014-12-10T11:50:29.349000Z],[2014-12-20T20:58:18.429000Z],[https://swapi.dev/api/planets/7/]
Wilhuff Tarkin,[180],[unknown],"[auburn, grey]",[fair],[blue],[64BBY],[male],[https://swapi.dev/api/planets/21/],"[[https://swapi.dev/api/films/1/, https://swap...",[[]],...,[polluted],[1 standard],[cityscape],[unknown],[22000000000],[[https://swapi.dev/api/people/12/]],[[]],[2014-12-10T16:26:54.384000Z],[2014-12-20T20:58:18.454000Z],[https://swapi.dev/api/planets/21/]
Yarael Poof,[264],[unknown],[none],[white],[yellow],[unknown],[male],[https://swapi.dev/api/planets/48/],[[https://swapi.dev/api/films/4/]],[[https://swapi.dev/api/species/25/]],...,[unknown],[unknown],[unknown],[unknown],[unknown],[[https://swapi.dev/api/people/57/]],[[]],[2014-12-20T10:34:08.249000Z],[2014-12-20T20:58:18.502000Z],[https://swapi.dev/api/planets/48/]


In [38]:
weather = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

In [39]:
weather

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.18400,NaN,NaN,NaN
1,2006-01-02,1380.52100,NaN,NaN,NaN
2,2006-01-03,1442.53300,NaN,NaN,NaN
3,2006-01-04,1457.21700,NaN,NaN,NaN
4,2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...,...
4378,2017-12-27,1263.94091,394.507,16.530,411.037
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714
